Esta solução pretende resolver um dos problemas propostos pelo professor como teste modelo

Especificamente, o problema contido no documento revisoes.pdf

Esta solução tentará ir ao máximo detalhe possivel para explicar o método de resolução passo por passo

Nota: esta solução foi criada por um aluno, não deve ser considerada como resolução oficial em nenhuma circunstancia, servindo mais como exercicio de revisão para o criador

In [1]:
import pandas as pd
import numpy as np

Primeiro passo: ler do ficheiro e obter os dados que pretendemos

In [2]:
df =  pd.read_csv(filepath_or_buffer='room-temperature.csv', header=None, sep=',')

In [3]:
X=df.loc[1:,1:4].values

Segundo passo: colocar os dados no formato correto

In [4]:
X.shape

(144, 4)

In [5]:
for i in range(0,144):
    for j in range(0,4):
        X[i][j]=float(X[i][j])

Nota: se não se adicionar o dtype no final o tipo será considerado como objeto em vez de float, e o que fizemos em cima será inutil

In [6]:
B = np.array(X,dtype=float)

Fazemos a transposição da matriz

Assim conseguimos que em vez de termos uma matriz 144 * 4 obtemos uma 4 * 144

Isto é importante pelo facto que a matriz B.BT(onde BT é a matriz B transposta) tem os mesmos valores próprios não nulos que a matriz BT.B, e as dimensões para este caso passam de 144 * 144 para 4 * 4

E sim, como a matriz está sobre dados numéricos podemos ver que a matriz tem 4 valores próprios não nulos

In [7]:
B=B.T

In [8]:
B.shape

(4, 144)

Se formos a verificar os dados de B vemos que eles não estão centrados em (0,0)
Para resolvermos isto, basta calcular a média e subtrair

In [9]:
media = np.mean(B,0)
phi=B-media

Com os dados centrados calculemos a matriz de covariância

In [10]:
mat_cov = np.cov(phi)
mat_cov

array([[ 0.6834076 ,  0.24726323, -0.48413331, -0.44653752],
       [ 0.24726323,  0.64830417, -0.47779226, -0.41777514],
       [-0.48413331, -0.47779226,  0.65795233,  0.30397324],
       [-0.44653752, -0.41777514,  0.30397324,  0.56033942]])

Em seguida vamos calcular os valores próprios e seus vetores próprios associados

In [11]:
valprop_sc, vect_prop_sc = np.linalg.eig(mat_cov)
valprop_sc, vect_prop_sc 

(array([ 1.82978200e+00, -1.91338430e-16,  4.19467288e-01,  3.00754230e-01]),
 array([[-0.511587  , -0.5       ,  0.69694553,  0.05045462],
        [-0.48724752, -0.5       , -0.70933841, -0.09710233],
        [ 0.5313029 , -0.5       ,  0.08045662, -0.67914944],
        [ 0.46753163, -0.5       , -0.06806374,  0.72579715]]))

Infelizmente o eig não ordena os valores nem os vetores próprios

Portanto temos de ser nós a fazer manualmente

In [12]:
idx = valprop_sc.argsort()[::-1]
valprop_sc = valprop_sc[idx]
vect_prop_sc = vect_prop_sc[:,idx]
valprop_sc,vect_prop_sc

(array([ 1.82978200e+00,  4.19467288e-01,  3.00754230e-01, -1.91338430e-16]),
 array([[-0.511587  ,  0.69694553,  0.05045462, -0.5       ],
        [-0.48724752, -0.70933841, -0.09710233, -0.5       ],
        [ 0.5313029 ,  0.08045662, -0.67914944, -0.5       ],
        [ 0.46753163, -0.06806374,  0.72579715, -0.5       ]]))

Como estávamos à espera, temos 4 valores próprios

Se olharmos para eles podemos ver que um deles à partida será retirado(ordem de grandeza imensamente inferior)

No entanto, devemos calcular própriamente se isto vai ocorrer ou não

Antes de mais, vamos calcular o traço, que é simplesmente o sumatório dos valores próprios da matriz

In [13]:
traco=sum(valprop_sc)

Com estes dados obtidos podemos finalmente fazer a primeira parte do exercicio: verificar se podemos reduzir dimensão ou não

Para tal podemos simplesmente correr um simples ciclo while

In [14]:
confianca_alvo=0.9
k=0
confianca=0
while confianca < confianca_alvo:
    confianca= confianca+valprop_sc[k]/traco
    k+=1
k, confianca

(3, 1.0)

Como estavamos à espera, podemos retirar um dos valores próprios

De facto neste caso o último contém tão pouca informação que a perda esperada é de 0.0

Assim está concluida a primeira parte do processo: determinar se se pode reduzir a dimensão da matriz de modo a que a perda da informação seja inferior a 90%

"Convenientemente", temos 3 valores próprios, pelo que temos 3 vectores próprios a eles associados, pelo que, como estamos a operar sobre valores reais, podemos projetar nos vetores e visualizar dita projecção(se quisermos)

Neste caso, não nos é pedida nenhuma imagem para visualizar a projecção, portanto não nos focaremos nisso

Primeiro vamos renomear as variáveis para algo ligeiramente mais compreensivel

In [15]:
l = valprop_sc
principal_axes  = vect_prop_sc
principal_components = phi.T.dot(principal_axes)


Vamos então reduzir a dimensão

In [17]:
PC_k = principal_components[:, 0:3]

In [21]:
PC_k.shape

(144, 3)

E .... está

A sério, é só isso

Em PC_k temos os valores das projeções dos vários pontos sobre os 3 vetores que calculamos